# SQLite

>[SQLite](https://en.wikipedia.org/wiki/SQLite) 是一个用 C 语言编写的数据库引擎。它不是一个独立的应用程序；相反，它是一个开发人员将其嵌入到应用程序中的库。因此，它属于嵌入式数据库系列。它是部署最广泛的数据库引擎，因为它是几个顶级网页浏览器、操作系统、手机和其他嵌入式系统所使用的。

在本指南中，我们将创建一个简单的对话链，它使用由 `SqliteEntityStore` 支持的 `ConversationEntityMemory`。

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## 用法

要使用存储，你只需提供两样东西：

1.  Session Id - 会话的唯一标识符，例如用户名、电子邮件、聊天 ID 等。
2.  Connection string - 一个指定数据库连接的字符串。对于 SQLite，该字符串是 `slqlite:///` 加上数据库文件的名称。如果该文件不存在，则会被创建。

In [1]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id="test_session_id", connection_string="sqlite:///sqlite.db"
)

chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hi")

In [2]:
chat_message_history.messages

[HumanMessage(content='Hello'), AIMessage(content='Hi')]

## 链式调用

我们可以轻松地将此消息历史类与 [LCEL Runnables](/docs/how_to/message_history) 结合使用。

为此，我们将使用 OpenAI，因此需要安装它。我们还需要将 OPENAI_API_KEY 环境变量设置为你的 OpenAI 密钥。

```bash
pip install -U langchain-openai

export OPENAI_API_KEY='sk-xxxxxxx'
```

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI()

In [5]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection_string="sqlite:///sqlite.db"
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [8]:
# This is where we configure the session id
config = {"configurable": {"session_id": "<SQL_SESSION_ID>"}}

In [9]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config)

AIMessage(content='Hello Bob! How can I assist you today?')

In [10]:
chain_with_history.invoke({"question": "Whats my name"}, config=config)

AIMessage(content='Your name is Bob! Is there anything specific you would like assistance with, Bob?')